# Confronto dati triggerless

In [44]:
!jt -t grade3 -fs 90  -tfs 15 -nfs 115 -cellw 80% -T
import ROOT as R
import array
import numpy as np
%jsroot on

In [40]:
c = R.TCanvas("c")
R.gStyle.SetOptStat(0);


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c


<hr>Faccio un Tchain per contenere tutti i dati nella cartella.

In [41]:
ch = R.TChain("tree")

In [45]:
ch.Add("./data_out/398/sub_*_G_*.root")

24

In [46]:
ch.Draw("timestamp:gemroc>>h(12,0,12,10000,0,10)", "subRunNo==3","colz")
c.Draw()

Il reset era stato dato via software, quindi mi sarei aspettato una differenza di qualche colpo di clock nei tempi di riferimento, ma non così sostanziale. <br> I pacchetti UDP ovviamente non arrivano in maniera sincrona, quindi non mi aspetto che il sistema cominci e finisca di acquisire tutto insieme

Proviamo ad esaminare il problema del buco. Se si zoomma nel plot sopra nel range di tempo intorno a 7.86s si nota che mancano dati per qualcosa più di 1ms in tutte le GEMROC. Il buco non è allineato, ma l'uguale durata fa pensare che sia un fenomeno comune a tutti il sistema. Bisogna capire se mancavano solo gli hit o anche le framewords. Ho messo in txt tutte le frameword in arrivo da gemroc 0. Vediamo dove cercare il buco.

In [49]:
ch.Draw("last_frame", "subRunNo==3 && timestamp>7.85 && timestamp<7.88","colz")
c.Draw()

C'è in effetti un buco nel file (lo metto come codice per non doverlo formattare):


In [53]:
# tiger 5 frame 38364
# tiger 3 frame 38364
# tiger 2 frame 38365
# tiger 7 frame 38365
# tiger 1 frame 38365
# tiger 5 frame 38365
# tiger 3 frame 38365
# tiger 6 frame 38365
# tiger 0 frame 38365
# tiger 6 frame 38366
# tiger 3 frame 38429
# tiger 7 frame 38429
# tiger 1 frame 38429
# tiger 4 frame 38429
# tiger 2 frame 38429
# tiger 6 frame 38429
# tiger 0 frame 38429
# tiger 0 frame 38430
# tiger 4 frame 38430

Bisogna fare altre prove per scoprire se questo buco è dovuto alla perdita di alcuni pacchetti (sarebbe strano però perdere lo un numero di pacchetti così alto, contemporaneamente, da tutto il sistema) o se è dovuto a qualche effetto sul clock distribuito. Inserendo nello script anche un controllo per le frame trovo che mancano in generale molto framewords. Bisognerà acquisire di nuovo e studiare meglio la cosa.